In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from __future__ import print_function, division

In [3]:
import os.path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
import os
import sys
base_module_path = os.path.abspath(os.path.join('..'))
if base_module_path not in sys.path:
    sys.path.append(base_module_path)
import ama as a

In [5]:
BASE_PATH = '../data/'

In [6]:
def check_image(path):
    print(path)
    img = plt.imread(path)
    print(img.shape)
    plt.imshow(img)

In [7]:
### COMPUTE FEATURES ###
from PIL import Image, ImageStat
import scipy
import scipy.stats
import cv2

def get_features(path):
    st = []
    img = Image.open(path)
    im_stats_ = ImageStat.Stat(img)
    st += im_stats_.sum
    st += im_stats_.mean
    st += im_stats_.rms
    st += im_stats_.var
    st += im_stats_.stddev
    img = np.array(img)[:,:,:3]
    st += [scipy.stats.kurtosis(img[:,:,0].ravel())]
    st += [scipy.stats.kurtosis(img[:,:,1].ravel())]
    st += [scipy.stats.kurtosis(img[:,:,2].ravel())]
    st += [scipy.stats.skew(img[:,:,0].ravel())]
    st += [scipy.stats.skew(img[:,:,1].ravel())]
    st += [scipy.stats.skew(img[:,:,2].ravel())]
    ### cv2 jpg ###
    bw = cv2.imread(path,0)
    st += list(cv2.calcHist([bw],[0],None,[256],[0,256]).flatten()) #bw 
    st += list(cv2.calcHist([img],[0],None,[256],[0,256]).flatten()) #r
    st += list(cv2.calcHist([img],[1],None,[256],[0,256]).flatten()) #g
    st += list(cv2.calcHist([img],[2],None,[256],[0,256]).flatten()) #b
    m, s = cv2.meanStdDev(img) #mean and standard deviation
    st += list(m.flatten())
    st += list(s.flatten())
    st += [cv2.Laplacian(bw, cv2.CV_64F).var()] 
    st += [cv2.Laplacian(img, cv2.CV_64F).var()]
    st += [cv2.Sobel(bw,cv2.CV_64F,1,0,ksize=5).var()]
    st += [cv2.Sobel(bw,cv2.CV_64F,0,1,ksize=5).var()]
    st += [cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5).var()]
    st += [cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5).var()]
    st += [(bw<30).sum()]
    st += [(bw>225).sum()]
    return [path, st]

In [8]:
from multiprocessing import Pool, cpu_count
def normalize_img(paths):
    imf_d = {}
    p = Pool(cpu_count())
    ret = p.map(get_features, paths)
    for i in range(len(ret)):
        imf_d[ret[i][0]] = ret[i][1]
    ret = []
    fdata = [imf_d[f] for f in paths]
    return fdata

In [9]:
import cPickle as pickle

In [10]:
import glob
test_jpg = glob.glob(BASE_PATH+ 'test-full-jpg/*'); len(test_jpg)
test = pd.DataFrame([[p.split('/')[3].replace('.jpg',''),p] for p in test_jpg]); test
test.columns = ['image_name','path']; test

,image_name,path
0,test_21294,../data/test-full-jpg/test_21294.jpg
1,test_26665,../data/test-full-jpg/test_26665.jpg
2,file_9697,../data/test-full-jpg/file_9697.jpg
3,test_35596,../data/test-full-jpg/test_35596.jpg
4,test_1065,../data/test-full-jpg/test_1065.jpg
5,test_25614,../data/test-full-jpg/test_25614.jpg
6,file_9410,../data/test-full-jpg/file_9410.jpg
7,test_20253,../data/test-full-jpg/test_20253.jpg
8,test_1288,../data/test-full-jpg/test_1288.jpg
9,file_18005,../data/test-full-jpg/file_18005.jpg


In [11]:
test['fdata'] = normalize_img(test['path'])

In [13]:
with open(BASE_PATH + 'df_test.pkl', 'wb') as output:
    pickle.dump(test, output, protocol=2)

In [20]:
test.head()

,image_name,path,fdata
0,test_21294,../data/test-full-jpg/test_21294.jpg,"[9070526.0, 8655941.0, 9441274.0, 0.0, 138.405..."
1,test_26665,../data/test-full-jpg/test_26665.jpg,"[12593353.0, 11807800.0, 12172622.0, 0.0, 192...."
2,file_9697,../data/test-full-jpg/file_9697.jpg,"[12506004.0, 11601409.0, 11873153.0, 0.0, 190...."
3,test_35596,../data/test-full-jpg/test_35596.jpg,"[13047340.0, 12318256.0, 12720091.0, 0.0, 199...."
4,test_1065,../data/test-full-jpg/test_1065.jpg,"[14297611.0, 13226229.0, 13484758.0, 0.0, 218...."


In [19]:
import xgboost as xgb
d_feat_test = xgb.DMatrix(np.vstack(test['fdata'].map(lambda x: np.array(x)).values))

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [25]:
model_paths = glob.glob(BASE_PATH+ 'xgb-models/*'); print(model_paths)
for p in model_paths:
    print(p.split('/')[3].split('-')[0])

['../data/xgb-models/slash_burn-model.pkl', '../data/xgb-models/cultivation-model.pkl', '../data/xgb-models/habitation-model.pkl', '../data/xgb-models/blooming-model.pkl', '../data/xgb-models/water-model.pkl', '../data/xgb-models/blow_down-model.pkl', '../data/xgb-models/bare_ground-model.pkl', '../data/xgb-models/selective_logging-model.pkl', '../data/xgb-models/cloudy-model.pkl', '../data/xgb-models/agriculture-model.pkl', '../data/xgb-models/artisinal_mine-model.pkl', '../data/xgb-models/primary-model.pkl', '../data/xgb-models/road-model.pkl', '../data/xgb-models/clear-model.pkl', '../data/xgb-models/partly_cloudy-model.pkl', '../data/xgb-models/haze-model.pkl', '../data/xgb-models/conventional_mine-model.pkl']
slash_burn
cultivation
habitation
blooming
water
blow_down
bare_ground
selective_logging
cloudy
agriculture
artisinal_mine
primary
road
clear
partly_cloudy
haze
conventional_mine


In [26]:
for p in model_paths:
    print(p)
    model = pickle.load(open(p, 'rb'))
    test[p.split('/')[3].split('-')[0]+'_pred'] = model.predict(d_feat_test)

../data/xgb-models/slash_burn-model.pkl
../data/xgb-models/cultivation-model.pkl
../data/xgb-models/habitation-model.pkl
../data/xgb-models/blooming-model.pkl
../data/xgb-models/water-model.pkl
../data/xgb-models/blow_down-model.pkl
../data/xgb-models/bare_ground-model.pkl
../data/xgb-models/selective_logging-model.pkl
../data/xgb-models/cloudy-model.pkl
../data/xgb-models/agriculture-model.pkl
../data/xgb-models/artisinal_mine-model.pkl
../data/xgb-models/primary-model.pkl
../data/xgb-models/road-model.pkl
../data/xgb-models/clear-model.pkl
../data/xgb-models/partly_cloudy-model.pkl
../data/xgb-models/haze-model.pkl
../data/xgb-models/conventional_mine-model.pkl


In [27]:
test.head()

,image_name,path,fdata,slash_burn_pred,cultivation_pred,habitation_pred,blooming_pred,water_pred,blow_down_pred,bare_ground_pred,selective_logging_pred,cloudy_pred,agriculture_pred,artisinal_mine_pred,primary_pred,road_pred,clear_pred,partly_cloudy_pred,haze_pred,conventional_mine_pred
0,test_21294,../data/test-full-jpg/test_21294.jpg,"[9070526.0, 8655941.0, 9441274.0, 0.0, 138.405...",0.000051,0.086791,0.060930,0.000055,0.467879,0.000007,0.004125,0.000342,0.000774,0.347777,1.316120e-05,0.996527,0.200731,0.079989,0.901145,0.151885,0.000068
1,test_26665,../data/test-full-jpg/test_26665.jpg,"[12593353.0, 11807800.0, 12172622.0, 0.0, 192....",0.000219,0.003408,0.000152,0.000587,0.005506,0.000031,0.002914,0.000403,0.000009,0.001769,1.438368e-06,0.999947,0.002156,0.998338,0.000063,0.001142,0.000003
2,file_9697,../data/test-full-jpg/file_9697.jpg,"[12506004.0, 11601409.0, 11873153.0, 0.0, 190....",0.000435,0.062570,0.017249,0.000509,0.021663,0.002063,0.018117,0.000231,0.001084,0.031242,2.701051e-06,0.999820,0.032696,0.001393,0.993383,0.000091,0.000004
3,test_35596,../data/test-full-jpg/test_35596.jpg,"[13047340.0, 12318256.0, 12720091.0, 0.0, 199....",0.000082,0.008315,0.001160,0.000334,0.026524,0.000121,0.001426,0.000276,0.000013,0.033047,7.638131e-07,0.999949,0.006555,0.998401,0.000744,0.000386,0.000004
4,test_1065,../data/test-full-jpg/test_1065.jpg,"[14297611.0, 13226229.0, 13484758.0, 0.0, 218....",0.000040,0.006656,0.001064,0.003787,0.138515,0.000940,0.000682,0.000418,0.000016,0.002178,1.154728e-06,0.999926,0.009164,0.999604,0.000471,0.000099,0.000005


In [28]:
test.drop(['path','fdata'], 1).to_csv(BASE_PATH + 'test_pred.csv', index=False)